# I. Project Team Members

| Prepared by | Email | Prepared for |
| :-: | :-: | :-: |
| **Hardefa Rogonondo** | hardefarogonondo@gmail.com | **Food.com Recipe Recommendation Engine** |

# II. Notebook Target Definition

_Insert Text Here_

# III. Notebook Setup

## III.A. Import Libraries

In [1]:
from scipy.sparse import csr_matrix
import numpy as np
import pandas as pd
import pickle

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## III.B. Import Data

In [2]:
train = pd.read_pickle('../../../data/processed/train.pkl')
test = pd.read_pickle('../../../data/processed/test.pkl')

In [3]:
train.head()

,user_id,recipe_id,date,rating,review,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
582596,897949,145965,2010-01-25,5,These were so easy. I tried other recipes whe...,sugared cinnamon almonds,35,52595,2005-11-22,"['60-minutes-or-less', 'time-to-make', 'course...","[1208.1, 108.0, 392.0, 4.0, 63.0, 26.0, 43.0]",11,"['preheat oven to 300 degrees', 'line a baking...",just like you find at the mall. a dear friend ...,"['whole almonds', 'sugar', 'cinnamon', 'egg wh...",6
1109913,962690,286212,2009-09-17,3,This smells so good cooking but I found it jus...,fakes traditional greek lentil soup,55,288218,2008-02-12,"['60-minutes-or-less', 'time-to-make', 'course...","[368.0, 26.0, 11.0, 1.0, 30.0, 12.0, 12.0]",8,['boil one glass of water and add the lentils'...,this is my mum's way for fakes. its a very com...,"['dry lentils', 'onion', 'garlic cloves', 'bay...",9
535345,383346,302395,2010-05-07,5,This is very yummy. I cooked the snow peas lo...,quick glazed snow peas,12,207176,2008-05-06,"['15-minutes-or-less', 'time-to-make', 'course...","[77.3, 3.0, 29.0, 14.0, 7.0, 6.0, 3.0]",4,"['place all ingredients , except the peas , in...","sun-sentinel, february 28, 2008. want to try ...","['butter', 'honey', 'chicken broth', 'salt', '...",5
278036,133174,185336,2007-06-27,4,This was very good. The dressing helps to ens...,honey dijon chicken and vegetables,435,86764,2006-09-08,"['time-to-make', 'course', 'main-ingredient', ...","[154.2, 2.0, 23.0, 3.0, 54.0, 1.0, 2.0]",10,"['rinse the chicken and pat dry', 'place in a ...",this recipe is a perfect dish to serve company...,"['boneless skinless chicken breasts', 'honey m...",8
731815,329769,425086,2010-05-21,4,*Reviewed for ZWT6 Whine and Cheese Gang* A g...,lychees ice cream with and without frangeli...,5,344231,2010-05-13,"['15-minutes-or-less', 'time-to-make', 'course...","[136.6, 11.0, 56.0, 2.0, 4.0, 22.0, 5.0]",5,"['so simple !', 'place ice cream in a dish', '...",a sublime dessert which i serve after a nice a...,"['vanilla ice cream', 'canned lychees', 'frang...",3


In [4]:
test.head()

,user_id,recipe_id,date,rating,review,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
900162,1378564,77397,2011-02-12,5,"very easy to make and bakes up great, i made i...",kittencal s 5 minute cinnamon flop brunch cake,25,89831,2003-11-27,"['30-minutes-or-less', 'time-to-make', 'course...","[291.1, 9.0, 142.0, 7.0, 7.0, 18.0, 18.0]",8,"['set oven to 350 degrees f', 'prepare a greas...",except for the the milk and small amount in th...,"['all-purpose flour', 'sugar', 'milk', 'vanill...",11
1108075,653438,432797,2012-10-30,5,What a great recipe for a quick dinner when yo...,simple yet delicious pizzas,2,1326655,2010-07-21,"['weeknight', '15-minutes-or-less', 'time-to-m...","[247.3, 15.0, 9.0, 20.0, 25.0, 25.0, 8.0]",5,"['toast english muffins', 'put a spoonful of p...",i made this recipe in college when i was low o...,"['english muffin', 'pizza sauce', 'mozzarella ...",4
404029,590135,10579,2008-06-17,5,This crust is so versatile! It is easy to make...,vegan pizza dough,40,157425,2001-08-01,"['60-minutes-or-less', 'time-to-make', 'course...","[229.2, 6.0, 0.0, 12.0, 16.0, 3.0, 13.0]",10,"['preheat oven to 400 degrees f', 'in large bo...",this is an updated version of my adopted recip...,"['flour', 'fast rising yeast', 'salt', 'water'...",5
866360,771469,32204,2008-04-25,5,"Finally the perfect brownie - not too ""cakey"" ...",whatever floats your boat brownies,35,37305,2002-06-25,"['60-minutes-or-less', 'time-to-make', 'course...","[390.7, 30.0, 161.0, 7.0, 12.0, 50.0, 17.0]",14,"['preheat oven to 350f', 'grease an 8 inch squ...","these are absolutely the chewiest, moistest, f...","['butter', 'unsweetened cocoa', 'sugar', 'eggs...",14
782401,748565,372474,2009-06-02,4,Very yummy and different salsa. I wasn't sure ...,the saucy senoritas salsa,45,80353,2009-05-17,"['60-minutes-or-less', 'time-to-make', 'course...","[135.7, 14.0, 30.0, 12.0, 3.0, 6.0, 4.0]",3,"['in a medium bowl , combine all the ingredien...","the name alone has you wanting some, doesn't i...","['mango', 'avocado', 'tomatoes', 'red bell pep...",14


# IV. Feature Engineering

## IV.A. Data Shape Inspection

In [5]:
train.shape, test.shape

((661320, 16), (283424, 16))

## IV.B. Data Information Inspection

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 661320 entries, 582596 to 287088
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   user_id         661320 non-null  object        
 1   recipe_id       661320 non-null  object        
 2   date            661320 non-null  datetime64[ns]
 3   rating          661320 non-null  int64         
 4   review          661308 non-null  object        
 5   name            661320 non-null  object        
 6   minutes         661320 non-null  int64         
 7   contributor_id  661320 non-null  object        
 8   submitted       661320 non-null  datetime64[ns]
 9   tags            661320 non-null  object        
 10  nutrition       661320 non-null  object        
 11  n_steps         661320 non-null  int64         
 12  steps           661320 non-null  object        
 13  description     661320 non-null  object        
 14  ingredients     661320 non-null  obj

In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 283424 entries, 900162 to 170451
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   user_id         283424 non-null  object        
 1   recipe_id       283424 non-null  object        
 2   date            283424 non-null  datetime64[ns]
 3   rating          283424 non-null  int64         
 4   review          283417 non-null  object        
 5   name            283424 non-null  object        
 6   minutes         283424 non-null  int64         
 7   contributor_id  283424 non-null  object        
 8   submitted       283424 non-null  datetime64[ns]
 9   tags            283424 non-null  object        
 10  nutrition       283424 non-null  object        
 11  n_steps         283424 non-null  int64         
 12  steps           283424 non-null  object        
 13  description     283424 non-null  object        
 14  ingredients     283424 non-null  obj

## IV.C. Unused Feature Removal

In [8]:
def unused_feat_removal(df, feature_to_remove):
    df.drop(columns=feature_to_remove, inplace=True)
    return df

In [9]:
feature_to_remove = ["date", "review", "name", "minutes", "contributor_id", "submitted",
                     "tags", "nutrition", "n_steps", "steps", "description", "ingredients", "n_ingredients"]

In [10]:
unused_feat_removal(train, feature_to_remove)
unused_feat_removal(test, feature_to_remove)
train.shape, test.shape

((661320, 3), (283424, 3))

In [11]:
train.head()

,user_id,recipe_id,rating
582596,897949,145965,5
1109913,962690,286212,3
535345,383346,302395,5
278036,133174,185336,4
731815,329769,425086,4


In [12]:
test.head()

,user_id,recipe_id,rating
900162,1378564,77397,5
1108075,653438,432797,5
404029,590135,10579,5
866360,771469,32204,5
782401,748565,372474,4


## IV.D. User-Item Matrix Creation

In [13]:
def correct_data_type(df_list):
    for df in df_list:
        df[["user_id", "recipe_id"]] = df[["user_id", "recipe_id"]].astype(int)


def create_mappings(train, test):
    user_mapping = {user: i for i, user in enumerate(
        train["user_id"].unique())}
    all_recipe_ids = np.union1d(
        train["recipe_id"].unique(), test["recipe_id"].unique())
    recipe_mapping = {recipe: i for i, recipe in enumerate(all_recipe_ids)}
    return user_mapping, recipe_mapping


def apply_mappings(df, user_mapping, recipe_mapping):
    df["user_id_mapped"] = df["user_id"].map(user_mapping).astype(int)
    df["recipe_id_mapped"] = df["recipe_id"].map(recipe_mapping).astype(int)
    return df


def create_sparse_user_item_matrix(df):
    rows = df["user_id_mapped"].values
    cols = df["recipe_id_mapped"].values
    data = df["rating"].values
    return csr_matrix((data, (rows, cols)))

In [14]:
correct_data_type([train, test])
user_mapping, recipe_mapping = create_mappings(train, test)
train = apply_mappings(train, user_mapping, recipe_mapping)
test = apply_mappings(test, user_mapping, recipe_mapping)
train_sparse_matrix = create_sparse_user_item_matrix(train)
test_sparse_matrix = create_sparse_user_item_matrix(test)

## IV.E. Final Feature Inspection

In [15]:
train_sparse_matrix.shape, test_sparse_matrix.shape

((59204, 214537), (59204, 214534))

In [16]:
train_dense_submatrix = train_sparse_matrix[:10, :10].todense()
train_submatrix = pd.DataFrame(train_dense_submatrix)
train_submatrix

,0,1,2,3,4,5,6,7,8,9
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0


In [17]:
test_dense_submatrix = train_sparse_matrix[:10, :10].todense()
test_submatrix = pd.DataFrame(test_dense_submatrix)
test_submatrix

,0,1,2,3,4,5,6,7,8,9
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0


## V.F. Export Data

In [18]:
with open('../../../data/processed/train_sparse_matrix.pkl', 'wb') as file:
    pickle.dump(train_sparse_matrix, file)

with open('../../../data/processed/test_sparse_matrix.pkl', 'wb') as file:
    pickle.dump(test_sparse_matrix, file)